In [1]:
import json
import numpy as np
import pandas as pd
import os

from glob import glob
from typing import Callable
from typing import Dict
from typing import List
from typing import Tuple

In [4]:
!wc -l ../PLArticlesScraper/PLArticlesScraper/scrapy_output/*.jsonl

    3806 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/bankier.jsonl
     789 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/bankier_moto.jsonl
    4702 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/bryla.jsonl
   12946 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/gazetakrakowska.jsonl
    5905 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/krakowwpigulce.jsonl
      93 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/mamstartup.jsonl
     356 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/naszemiasto.jsonl
   67651 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/nowiny24.jsonl
    5280 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/nowymarketing.jsonl
   25686 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/polki.jsonl
   29133 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/spidersweb.jsonl
   78389 ../PLArticlesScraper/PLArticlesScraper/scrapy_output/sport.jsonl
    3536 ../PLArticlesScraper/PLArticlesScraper/scrapy_ou

In [6]:
class FilesReader:
    def __init__(self, data_paths: str):
        self.data_paths = data_paths
        self.files = glob(data_paths + '*')
        self.files.sort()
        self._get_file_names()
        self.data = None
        
    def _get_file_names(self):
        self.file_names = [
            os.path.basename(file).split('.')[0]
            for file in self.files
        ]
        
    def read_json_lines_files(self) -> None:
        self.data = dict(zip(self.file_names, (pd.DataFrame(self._read_json_lines_file(file)) for file in files)))
        
    def _read_json_lines_file(self, file: str) -> list:
        list_of_lines = []
        with open(file, 'r', encoding='utf-8') as opened_file:
            for line in opened_file:
                list_of_lines.append(json.loads(line.rstrip('\n|\r')))
        return list_of_lines

In [27]:
class Filterer:
    MIN_MAX_LENGTH = (5, 100)
    
    def __init__(
        self, 
        title_lead_length_limit: Tuple[int, int]=MIN_MAX_LENGTH, 
        text_length_limit: Tuple[int, int]=MIN_MAX_LENGTH, 
        reset_index: bool=True,
    ):
        self.title_lead_length_limit = title_lead_length_limit
        self.text_length_limit = text_length_limit
        self.column_types = ['header', 'text']
        self.default_header_columns = ['title', 'lead']
        self.default_text_columns = ['text']
        self.reset_index=True
        self.set_conditions(title_lead_length=title_lead_length_limit, text_length=text_length_limit)
        
    def get_default_columns(self, skip_title: bool=True) -> Dict[str, List[str]]:
        header_columns = self.default_header_columns
        if skip_title:
            header_columns = [header for header in self.default_header_columns if header != 'title']
        return dict(zip(self.column_types, (header_columns, self.default_text_columns)))
    
    def get_default_conditions(self) -> Dict[str, Callable]:
        return dict(zip(self.column_types, self.conditions)) 
    
    def set_conditions(
        self, 
        title_lead_length: Tuple[int, int]=MIN_MAX_LENGTH, 
        text_length: Tuple[int, int]=MIN_MAX_LENGTH,
    ) -> None:
        self.conditions = [
            self.set_condition(title_lead_length), self.set_condition(text_length)
        ]
        
    def set_condition(self, length: Tuple[int, int]) -> Callable:
        return lambda x: max(length[0], 1) <= len(" ".join(x).strip().split()) <= length[1] \
            if isinstance(x, list) else max(length[0], 1) <= len(x.strip().split()) <= length[1]
        
    def filter_site_data(
        self, 
        data: pd.DataFrame, 
        columns_names: Dict[str, List[str]]=None, 
        conditions: Dict[str, Callable]=None,
        skip_title: bool=True,
    ) -> pd.DataFrame:
        if not columns_names:
            columns_names = self.get_default_columns()
        if not conditions:
            conditions = self.get_default_conditions()

        for column_type, col_names in columns_names.items():
            column_condition = conditions[column_type]
            for col_name in col_names:
                data = data[data[col_name].map(column_condition)]
        if self.reset_index:
            return data.reset_index(drop=True)
        return data

In [7]:
file_reader = FilesReader(data_paths='../PLArticlesScraper/PLArticlesScraper/scrapy_output/')

In [11]:
file_reader.read_json_lines_files()

In [15]:
file_reader.data['bankier'].head()

,url,title,lead,text,text_main_points
0,https://www.bankier.pl/wiadomosc/Oszustwo-SMS-...,"[""Pieniądze z Państwa konta przekazane do NBP""...",[Największy bank w Polsce bije na alarm: oszuś...,"[""Przekażcie dalej i ostrzeżcie znajomych"" - a...",[]
1,https://www.bankier.pl/wiadomosc/Zakaz-handlu-...,[Zakaz handlu zawieszony z powodu koronawirusa...,[Zawieszenie wejścia w życie wszelkich ustaw w...,[\r\nJak czytamy w stanowisku KIG przysłanym w...,[]
2,https://www.bankier.pl/wiadomosc/Czechy-zamyka...,[Czechy zamykają sklepy i restauracje],[Władze Republiki Czeskiej poinformowały w noc...,[Otwarte będą natomiast wszystkie stacje benzy...,[]
3,https://www.bankier.pl/wiadomosc/5-tematow-min...,[5 tematów mijającego tygodnia ważnych dla two...,[W mijającym tygodniu\r\nopisywaliśmy szereg t...,[W ramach cyklu „5\r\ntematów minionego tygodn...,[]
4,https://www.bankier.pl/wiadomosc/Turczynowicz-...,[Turczynowicz-Kieryłło rezygnuje funkcji szefo...,[Z uwagi na stan zdrowia nie będę pełnić już f...,"[\r\n, ""Moi Drodzy, z uwagi na stan zdrowia ni...",[]


In [14]:
for key in file_reader.data.keys():
    print(f'{key} : {len(file_reader.data[key])}')

bankier : 3806
bankier_moto : 789
bryla : 4702
gazetakrakowska : 12946
krakowwpigulce : 5905
mamstartup : 93
naszemiasto : 356
nowiny24 : 67651
nowymarketing : 5280
polki : 25686
spidersweb : 29133
sport : 78389
strefainwestorow : 3536
sztukaarchitektury : 3939
wirtualnemedia_biznes : 6664
wnp : 476


In [28]:
filterer = Filterer(title_lead_length_limit=(15, 50), text_length_limit=(50, 1000))

In [30]:
test_filtered = filterer.filter_site_data(file_reader.data['gazetakrakowska'])
test_filtered.head()

,url,title,lead,text
0,https://gazetakrakowska.pl/jordanow-w-nocy-spl...,[Jordanów. W nocy spłonął tartak. Niemal 70 st...,[Nocny pożar w Jordanowie w powiecie suskim. W...,[Ogień został zauważony o godz. 3.30 w nocy. N...
1,https://gazetakrakowska.pl/koronawirus-w-nowym...,[Koronawirus w Nowym Sączu. Wracający z zagran...,"[Sądeczanie, którzy wracają do domu z zagranic...",[Wielu Sądeczan pracujących zagranicą w związk...
2,https://gazetakrakowska.pl/krakow-trwa-budowa-...,[Kraków. Trwa budowa wodnego placu zabaw w par...,[Postępują prace związane z budową wodnego pla...,"[Wodny , plac zabaw, w parku Jordana to jedno..."
3,https://gazetakrakowska.pl/koronawirus-tarnow-...,[Koronawirus Tarnów. Wracający z zagranicy nie...,"[Mieszkańcy Tarnowa i regionu, którzy planują ...",[Pani Katarzyna spod Tarnowa pracowała dotąd w...
4,https://gazetakrakowska.pl/koronawirus-francis...,[Koronawirus. Franciszkanie z Krakowa zachęcaj...,"[W trakcie walki z epidemią koronawirusa, w Po...","[Powstają różne inicjatywy, jak wspierać tych,..."


### Max

In [31]:
test_filtered.applymap(lambda x: len(" ".join(x).strip().split()) if isinstance(x, list) else len(x.strip().split())).max()

url        1
title     26
lead      50
text     999
dtype: int64

### Min

In [32]:
test_filtered.applymap(lambda x: len(" ".join(x).strip().split()) if isinstance(x, list) else len(x.strip().split())).min()

url       1
title     0
lead     15
text     50
dtype: int64

# Filter data

In [35]:
data_filtered = {}

In [36]:
for site in file_reader.file_names:
    data_filtered[site] = filterer.filter_site_data(file_reader.data[site])

In [37]:
from IPython.display import display

In [56]:
for site in file_reader.file_names:
    display(site, len(data_filtered[site]), data_filtered[site].head())

'bankier'

2743

,url,title,lead,text,text_main_points
0,https://www.bankier.pl/wiadomosc/Oszustwo-SMS-...,"[""Pieniądze z Państwa konta przekazane do NBP""...",[Największy bank w Polsce bije na alarm: oszuś...,"[""Przekażcie dalej i ostrzeżcie znajomych"" - a...",[]
1,https://www.bankier.pl/wiadomosc/Zakaz-handlu-...,[Zakaz handlu zawieszony z powodu koronawirusa...,[Zawieszenie wejścia w życie wszelkich ustaw w...,[\r\nJak czytamy w stanowisku KIG przysłanym w...,[]
2,https://www.bankier.pl/wiadomosc/Czechy-zamyka...,[Czechy zamykają sklepy i restauracje],[Władze Republiki Czeskiej poinformowały w noc...,[Otwarte będą natomiast wszystkie stacje benzy...,[]
3,https://www.bankier.pl/wiadomosc/5-tematow-min...,[5 tematów mijającego tygodnia ważnych dla two...,[W mijającym tygodniu\r\nopisywaliśmy szereg t...,[W ramach cyklu „5\r\ntematów minionego tygodn...,[]
4,https://www.bankier.pl/wiadomosc/Turczynowicz-...,[Turczynowicz-Kieryłło rezygnuje funkcji szefo...,[Z uwagi na stan zdrowia nie będę pełnić już f...,"[\r\n, ""Moi Drodzy, z uwagi na stan zdrowia ni...",[]


'bankier_moto'

109

...

4246

,url,title,lead,text
0,https://www.wirtualnemedia.pl/artykul/dominika...,[Dominika Kulczyk przekaże 20 mln zł na walkę ...,[Dominika Kulczyk zdecydowała się przekazać Fu...,[Środki zostaną przekazane na zakup: sprzętu d...
1,https://www.wirtualnemedia.pl/artykul/koronawi...,[Maspex przekaże szpitalom zakaźnym 1 mln swoi...,"[Firma spożywcza Maspex, do której należą m.in...","[Maspex wyliczył, że przekaże szpitalom zakaźn..."
2,https://www.wirtualnemedia.pl/artykul/colian-p...,[Colian przekaże słodycze i napoje szpitalom p...,[Spółka Colian (właściciel takich marek jak He...,"[Spółka , Colian , zadeklarowała wysyłkę słody..."
3,https://www.wirtualnemedia.pl/artykul/zus-wpro...,[ZUS wprowadza ułatwienia dla firm. Konfederac...,[Nie czekając na decyzje rządu w sprawie pakie...,"[Zakład Ubezpieczeń Społecznych poinformował, ..."
4,https://www.wirtualnemedia.pl/artykul/seniorzy...,[Seniorzy w Orange z wydłużonym terminem płatn...,"[Operator Orange Polska zapowiedział, że jego ...",[Wprowadzając ulgi w rozliczeniach Orange Pols...


'wnp'

140

,url,title,lead,text,text_main_points
0,https://www.wnp.pl/energetyka/pge-wobec-korona...,[PGE wobec koronawirusa. Gwarantuje dostawy pr...,[Polska Grupa Energetyczna (PGE) 13 marca zape...,"[\n , \n\n , \n\n\t\n\t\n Spółka odpo...",[]
1,https://www.wnp.pl/przemysl-obronny/koronawiru...,[Koronawirus dopadł polską armię],"[Żarty się skończyły, zaczęły się schody. W Po...","[\n , \n\n , \n\n\t\n\t\n , \nTo było...","[Koronawirus dopadł polską armię, gen. Jarosła..."
2,https://www.wnp.pl/hutnictwo/arcelormittal-wzy...,[ArcelorMittal wzywa rządy UE do poparcia węgl...,"[ArcelorMittal wezwał państwa członkowskie, by...","[\n , \n\n , \n\n\t\n\t\n Według Arce...",[]
3,https://www.wnp.pl/hutnictwo/cla-trumpa-zamyka...,[Cła Trumpa zamykają amerykańską walcownię],[Walcownia NLMK Pensylwania musi kupować stal ...,"[\n , \n\n , \n\n\t\n\t\n Walcownia N...",[]
4,https://www.wnp.pl/nieruchomosci/slaskie-kamie...,[Śląskie Kamienice wchodzą na rynek łódzki],[Śląskie Kamienice opracowały strategię rozwoj...,"[\n , \n\n , \n\n\t\n\t\n Śląskie Kam...",[]


In [58]:
sum([len(data_filtered[site]) for site in file_reader.file_names])

158749